# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
import time, numpy as np, matplotlib.pyplot as plt, pandas as pd

import gimbal_control as gc
from extended_kalman_filter import ExtendedKalmanFilter
from image_classifier import ImageClassifier

In [ ]:
def plot_util(_plt):
    _plt.legend()
    _plt.grid()
    fig = _plt.gcf()
    fig.set_size_inches(18.5, 5, forward=True)
    _plt.show()

In [ ]:
class ExperimentLogger():
    def __init__(self):
        self.phi_yaw_arr = []
        self.EKF_yaw_arr = []
        self.gc_yaw_arr = []
        
        self.phi_pitch_arr = []
        self.EKF_pitch_arr = []
        self.gc_pitch_arr = []
        
        self.time_arr = []
    
    def log(self, gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, t):
        self.phi_yaw_arr.append(phi_yaw)
        self.EKF_yaw_arr.append(EKF_yaw.get_pos())

        self.phi_pitch_arr.append(phi_pitch)
        self.EKF_pitch_arr.append(EKF_pitch.get_pos())

        self.gc_yaw_arr.append(gc_angles['yaw'])
        self.gc_pitch_arr.append(gc_angles['pitch'])
        
        self.time_arr.append(t)

## A Basic Loop

In [ ]:
# setup stuff
EKF_yaw = ExtendedKalmanFilter(Ts=0.05, Q=0.5, R=0.005, a=0.94)    # need to tweak Q and R
EKF_pitch = ExtendedKalmanFilter(Ts=0.05, Q=0.5, R=0.005, a=0.94)  # need to tweak Q and R
IC = ImageClassifier(
    graph_filename='../Models/MobileNet_SSD_caffe/graph',
    label_filename='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person',
    camera_resolution=(1640,922),
    debug=False)

# constants
total_run_time = 20 # seconds
t_loop_desired = 0.05       # seconds = 50 ms

# get EKFs trained up on the initial angles
initial_angles = gc.get_motor_angles()
for i in range(10):
    EKF_yaw.predict(); EKF_yaw.update(initial_angles['yaw'])
    EKF_pitch.predict(); EKF_pitch.update(initial_angles['pitch'])

# logging used for offline stuff only
EL = ExperimentLogger()

# a rolling buffer for the gimbal angles
gc_logger_len = 15
gc_logger_time = np.zeros(gc_logger_len)
gc_logger_yaw = np.zeros(gc_logger_len)
gc_logger_pitch = np.zeros(gc_logger_len)
gc_i = 0

# initialize variables in case the neural net doesn't spot anything the first time round
phi_yaw, phi_pitch, idx = 0, 0, 0

In [ ]:
while IC.dict_queue.empty() is True: pass # wait until there's a result from the nn

t_start = time.time()
t_since_nn_update = time.time()
while True:
    t = time.time()
    print('+', end='')
    
    ######################### LOG THE GIMBAL ANGLES #########################
    gc_angles = gc.get_motor_angles()
    gc_angles['yaw'] = ((gc_angles['yaw']+90) % 180)-90 # get rid of erroneous spikes
    gc_angles['pitch'] = ((gc_angles['pitch']+90) % 180)-90

    gc_logger_time[gc_i % gc_logger_len] = time.time()
    gc_logger_yaw[gc_i % gc_logger_len] = gc_angles['yaw']
    gc_logger_pitch[gc_i % gc_logger_len] = gc_angles['pitch']
    gc_i += 1
    
    ######################### NN and EKF #########################
#     EKF_yaw.predict()
#     EKF_pitch.predict()
    
    # if there is a new result waiting...
    if not IC.dict_queue.empty():
        print('.',end='')
        bb, bb_angles, photo_time = IC.get_result()

        # and there was an actual object detected...
        if bb_angles != -1:
            (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
            phi_yaw = (phi_x1 + phi_x2)/2
            phi_pitch = (phi_y1 + phi_y2)/2

            idx = np.argmin(np.abs(gc_logger_time - photo_time)) # time closest to the photo time

            EKF_yaw.predict()
            EKF_pitch.predict()
            EKF_yaw.update(phi_yaw + gc_logger_yaw[idx])
            EKF_pitch.update(phi_pitch + gc_logger_pitch[idx])
            
            t_since_nn_update = time.time()# TODO: this might not be the best approach! maybe do photo_time??

#             print('time diff = %d [ms]' % ((time.time() - gc_logger_time[idx])*1000))
    
    ######################### CONTROL #########################
#     desired_yaw = EKF_yaw.get_pos()
    desired_yaw = EKF_yaw.predict_ahead(time.time() - t_since_nn_update)
    gc.send_angle_command(0, 0, desired_yaw)  # NOTE: NO PITCH
    
    ######################### TIMING #########################    
    t_end = time.time()
    if t_end > t_start + total_run_time:
        break
    elif t_end - t > t_loop_desired:
        EL.log(gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, time.time() - t_start)
    else:
        time.sleep(t_loop_desired - (t_end - t))  # aim for a loop time of 50ms
        EL.log(gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, time.time() - t_start)

IC.close()
gc.send_angle_command(0, 0, 0);

In [ ]:
IC.close()
gc.send_angle_command(0, 0, 0);

In [ ]:
# df = pd.DataFrame(
#     columns=['phi_yaw_arr', 'phi_pitch_arr',
#              'gc_yaw_arr', 'gc_pitch_arr',
#              'time_arr'],
#     data=np.array([EL.phi_yaw_arr, EL.phi_pitch_arr,
#                    EL.gc_yaw_arr, EL.gc_pitch_arr,
#                    EL.time_arr]).T)
# df.to_csv('logged_data_3.csv')

In [ ]:
plt.plot(EL.time_arr, EL.EKF_yaw_arr, label='EKF estimate of yaw [deg]')
plt.plot(EL.time_arr, EL.phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
# tmp = [min(100, i) for i in EL.gc_yaw_arr]
# tmp = [max(-100, i) for i in tmp]
# plt.plot(EL.time_arr, tmp, label='Gimbal yaw [deg]')
plt.plot(EL.time_arr, np.array(EL.gc_yaw_arr) + np.array(EL.phi_yaw_arr), label='Gimbal yaw [deg]')
plot_util(plt)

In [ ]:
plt.plot(EL.time_arr, EL.EKF_pitch_arr, label='EKF estimate of pitch [deg]')
plt.plot(EL.time_arr, EL.phi_pitch_arr, label='Raw NN estimate of pitch [deg]')
plt.plot(EL.time_arr, EL.gc_pitch_arr, label='Gimbal pitch [deg]')
plot_util(plt)

In [ ]:
loop_times_ms = [(t-t_)*1e3 for t_,t in zip(EL.time_arr[0:-1], EL.time_arr[1:])]
plt.stem(EL.time_arr[:-1], loop_times_ms, label='loop times [ms]')
plot_util(plt)

In [ ]:
plt.plot(EL.time_arr[20:len(EL.phi_yaw_arr)//2],
         EL.phi_yaw_arr[20:len(EL.phi_yaw_arr)//2], label='Raw NN estimate of yaw [deg]')
plot_util(plt)